In [ ]:
# ===============================================================================
# 3. ANÁLISE EXPLORATÓRIA DE DADOS (EDA)
# ===============================================================================

def perform_eda(df):
    """Análise exploratória completa"""
    
    print("\n📊 ANÁLISE EXPLORATÓRIA DE DADOS")
    print("="*50)
    
    # Configurar subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=('Distribuição da Produção', 'Correlações com Produção',
                       'Nutrientes vs Produção', 'Precipitação vs Produção',
                       'Distribuição de Nutrientes', 'Eficiência Hídrica'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 1. Distribuição da variável target
    fig.add_trace(
        go.Histogram(x=df['RICE_PRODUCTION'], name='Produção de Arroz', 
                    marker_color='lightblue', opacity=0.7),
        row=1, col=1
    )
    
    # 2. Correlações principais
    correlations = df[['ANNUAL', 'avg_rain', 'Nitrogen', 'POTASH', 'PHOSPHATE', 'RICE_PRODUCTION']].corr()['RICE_PRODUCTION'].drop('RICE_PRODUCTION')
    
    fig.add_trace(
        go.Bar(x=correlations.index, y=correlations.values, 
               name='Correlação com Produção',
               marker_color=['red' if x < 0 else 'green' for x in correlations.values]),
        row=1, col=2
    )
    
    # 3. Scatter: Nutrientes vs Produção
    fig.add_trace(
        go.Scatter(x=df['NPK_Total'], y=df['RICE_PRODUCTION'],
                  mode='markers', name='NPK Total vs Produção',
                  marker=dict(size=8, color=df['avg_rain'], 
                            colorscale='Viridis', showscale=True,
                            colorbar=dict(title="Chuva Média"))),
        row=2, col=1
    )
    
    # 4. Scatter: Precipitação vs Produção
    fig.add_trace(
        go.Scatter(x=df['ANNUAL'], y=df['RICE_PRODUCTION'],
                  mode='markers', name='Precipitação vs Produção',
                  marker=dict(size=8, color='orange')),
        row=2, col=2
    )
    
    # 5. Box plot de nutrientes
    nutrients = ['Nitrogen', 'POTASH', 'PHOSPHATE']
    for i, nutrient in enumerate(nutrients):
        fig.add_trace(
            go.Box(y=df[nutrient], name=nutrient, 
                  marker_color=px.colors.qualitative.Set1[i]),
            row=3, col=1
        )
    
    # 6. Eficiência Hídrica
    fig.add_trace(
        go.Scatter(x=df['Water_Efficiency'], y=df['RICE_PRODUCTION'],
                  mode='markers', name='Eficiência Hídrica',
                  marker=dict(size=10, color='purple')),
        row=3, col=2
    )
    
    # Atualizar layout
    fig.update_layout(height=1200, showlegend=True, 
                     title_text="Análise Exploratória - Produção de Arroz")
    fig.show()
    
    # Matriz de Correlação
    plt.figure(figsize=(12, 10))
    correlation_matrix = df.select_dtypes(include=[np.number]).corr()
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdYlBu_r', 
                center=0, square=True, fmt='.2f')
    plt.title('Matriz de Correlação', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return correlations

# ===============================================================================
# 4. ANÁLISE DE OUTLIERS
# ===============================================================================

def detect_outliers(df):
    """Detecta e analisa outliers"""
    
    print("\n🎯 DETECÇÃO DE OUTLIERS")
    print("="*30)
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    outliers_summary = {}
    
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outliers_summary[col] = len(outliers)
        
        if len(outliers) > 0:
            print(f"⚠️  {col}: {len(outliers)} outliers detectados")
    
    # Visualizar outliers
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    main_cols = ['RICE_PRODUCTION', 'Nitrogen', 'POTASH', 'PHOSPHATE', 'ANNUAL', 'avg_rain']
    
    for i, col in enumerate(main_cols):
        if i < len(axes):
            df.boxplot(column=col, ax=axes[i])
            axes[i].set_title(f'Outliers: {col}')
            axes[i].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.suptitle('Análise de Outliers por Variável', fontsize=16, y=1.02)
    plt.show()
    
    return outliers_summary

# ===============================================================================
# 5. PREPROCESSAMENTO
# ===============================================================================

def preprocess_data(df):
    """Preprocessa dados para machine learning"""
    
    print("\n🔄 PREPROCESSAMENTO DOS DADOS")
    print("="*40)
    
    # Separar features numéricas
    numeric_features = df.select_dtypes(include=[np.number]).columns.drop('RICE_PRODUCTION')
    target = 'RICE_PRODUCTION'
    
    X = df[numeric_features]
    y = df[target]
    
    print(f"Features selecionadas: {len(numeric_features)}")
    print(f"Samples: {len(X)}")
    
    # Split treino/teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=None
    )
    
    # Normalização
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=numeric_features, index=X_train.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=numeric_features, index=X_test.index)
    
    print("✅ Preprocessamento concluído!")
    print(f"Treino: {X_train_scaled.shape}, Teste: {X_test_scaled.shape}")
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler